One way to make text classification multilingual is to develop multilingual word embeddings. With this technique, embeddings for every language exist in the same vector space, and maintain the property that words with similar meanings (regardless of language) are close together in vector space. For example, the words futbol in Turkish and soccer in English would appear very close together in the embedding space because they mean the same thing in different languages.

__Goal:__ Inducing word translations using only monolingual corpora for two languages. Separate embeddings are trained for each language and a mapping is learned though an adversarial objective, along with an orthogonality constraint on the most frequent words.

Mapping can be obtained by using MUSE package:

First download the monolingual word embeddings:

curl -Lo data/wiki.en.vec https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec

curl -Lo data/wiki.fr.vec https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec

Then run the mapping:
python supervised.py --src_lang en --tgt_lang fr --src_emb data/wiki.en.vec --tgt_emb data/wiki.fr.vec --n_refinement 5 --dico_train default --cuda False

As an output (best_mapping.pth), I got the 300-dimension matrix, which is a translation matrix. Thus, given a point in a vector space for a word in English, and multiplying it with the translation matrix will give the point in the vector space for the corresponding word in French.

## Ideas to implement it:
### First idea: Using monolingual embeddings
- Get the representation of the document in English
- Find each token from the document, inside the monolingual embedding file (wiki.en.vec) and get the coefficient for that.
- Create matrix with vocabulary size and 300 dimensions with all zeros
- Looping over the words of a given document, for each iteration, (or for each word) set the matrix element (created with all zeros) to the coefficient obtained from wiki.en.vec as described in step 2.
- At the end, multiply the created matrix with the translation matrix from MUSE package (created as explained at the beginning).
- Use that matrix as weights in the Embedding layer.
- The final prediction can be found in the vector space given for French.


### Second idea: Using en-fr dictionary
- Get the representation of the document in English
- For each English word, look at the en-fr dictionary and find the corresponding French word
- Train documents will be the English words obtained from the given document, whereas labels will be the French words. 
- The translation matrix will be fed to the Embedding layer in Keras as weights

In [5]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
import torch 

emb_en_fr = torch.load("/Users/tulinvarol/MUSE/dumped/debug/x7og08cwhk/best_mapping.pth")
dict_en_fr = open("/Users/tulinvarol/MUSE/data/crosslingual/dictionaries/en-fr.txt")

word_en = []
word_fr = []
embeddings_index = {}
for line in dict_en_fr:
    values = line.split(' ')
    word_en.append(values[0])
    word_fr.append(values[1].strip())
    embeddings_index
dict_en_fr.close()